# Imports

In [1]:
from numpy.random import seed
seed(1)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import itertools

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline

import tensorflow 

from tensorflow import keras
from keras.layers import Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D
from keras.layers import Dropout

import scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.utils import to_categorical

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


# Load Dataset

In [2]:
lesion_type_dict = {'akiec': 'Actinic keratoses',
                    'bcc': 'Basal cell carcinoma',
                    'bkl': 'Benign keratosis-like lesions ',
                    'df': 'Dermatofibroma',
                    'nv': 'Melanocytic nevi',
                    'mel': 'Melanoma',
                    'vasc': 'Vascular lesions'}

path1 = "C:\\Users\\THANOS\\Desktop\\DSDM\\PERIOD 4\\DF\\ASSIGNMENT\\dataverse_files\\HAM10000_images_part_1" # HAM10000_images_part_1 path
path2 = "C:\\Users\\THANOS\\Desktop\\DSDM\\PERIOD 4\\DF\\ASSIGNMENT\\dataverse_files\\HAM10000_images_part_2" # HAM10000_images_part_2 path
path3 = "C:\\Users\\THANOS\\Desktop\\DSDM\\PERIOD 4\\DF\\ASSIGNMENT\\dataverse_files\\HAM10000_metadata.csv" # HAM10000_metadata.csv path

imageid_path_dict = {}
for path in [path1, path2]:
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".jpg"):
                imageid_path_dict[os.path.splitext(file)[0]] = os.path.join(root, file)

skin_df = pd.read_csv(path3)
skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get)
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes
skin_df.groupby(['dx']).count()

,lesion_id,image_id,dx_type,age,sex,localization,dataset,path,cell_type,cell_type_idx
dx,,,,,,,,,,
akiec,327,327,327,327,327,327,327,327,327,327
bcc,514,514,514,514,514,514,514,514,514,514
bkl,1099,1099,1099,1089,1099,1099,1099,1099,1099,1099
df,115,115,115,115,115,115,115,115,115,115
mel,1113,1113,1113,1111,1113,1113,1113,1113,1113,1113
nv,6705,6705,6705,6660,6705,6705,6705,6705,6705,6705
vasc,142,142,142,142,142,142,142,142,142,142


In [3]:
skin_df

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Benign keratosis-like lesions,2
...,...,...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Actinic keratoses,0
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Actinic keratoses,0
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Actinic keratoses,0
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,C:\Users\THANOS\Desktop\DSDM\PERIOD 4\DF\ASSIG...,Actinic keratoses,0


In [4]:
# TODO: Make sure there are no missing values or NaNs in your input data

skin_df['age'].fillna(int(skin_df['age'].mean()),inplace=True)

unknown_indices = skin_df[skin_df['sex'] == 'unknown'].index
unknown_count = len(unknown_indices)
replacement_values = np.random.choice(['male', 'female'], unknown_count)
skin_df.loc[unknown_indices, 'sex'] = replacement_values

In [5]:
skin_df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age              0
sex              0
localization     0
dataset          0
path             0
cell_type        0
cell_type_idx    0
dtype: int64

In [6]:
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((75,100))))
X=skin_df.drop(columns=['cell_type_idx'],axis=1)
y=skin_df['cell_type_idx']

# Splitting Dataset

In [7]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(X, y, test_size=0.2,random_state=666)
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())
x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)
x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)
x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 999)

#Reshaping the Images into 3 channels (RGB)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [8]:
input_shape = x_train.shape[1:]
#input_shape = (75, 100, 3)
num_classes = 7
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=4, verbose=1, factor=0.5, min_lr=0.00001)

#Data Generation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(x_train)

# Basic CNN model

In [9]:
b_cnn = Sequential()
b_cnn.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
b_cnn.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
b_cnn.add(AveragePooling2D(pool_size = (2, 2)))
b_cnn.add(Dropout(0.25))
b_cnn.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
b_cnn.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
b_cnn.add(AveragePooling2D(pool_size=(2, 2)))
b_cnn.add(Dropout(0.25))
b_cnn.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
b_cnn.add(AveragePooling2D(pool_size = (2, 2)))
b_cnn.add(Dropout(0.25))
b_cnn.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
b_cnn.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
b_cnn.add(AveragePooling2D(pool_size=(2, 2)))
b_cnn.add(Dropout(0.25))
b_cnn.add(Flatten())
b_cnn.add(Dense(128, activation='relu'))
b_cnn.add(Activation('relu'))
b_cnn.add(Dropout(0.25))
b_cnn.add(Dense(num_classes, activation='softmax'))
b_cnn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 75, 32)       896       
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 75, 64)       18496     
                                                                 
 average_pooling2d (Average  (None, 50, 37, 64)        0         
 Pooling2D)                                                      
                                                                 
 dropout (Dropout)           (None, 50, 37, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 37, 64)        36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 37, 64)        36928     
                                                       

In [10]:
b_cnn.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
b_cnn_history = b_cnn.fit_generator(datagen.flow(x_train,y_train, batch_size=16),
                              epochs = 50, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // 16
                              , callbacks=[learning_rate_reduction])

# TODO: check if with alternative datagen if the error still exists

C:\Users\THANOS\AppData\Local\Temp\ipykernel_11976\3277338296.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  b_cnn_history = b_cnn.fit_generator(datagen.flow(x_train,y_train, batch_size=16),


Epoch 1/50



ValueError: in user code:

    File "C:\Users\THANOS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\THANOS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\THANOS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\THANOS\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\THANOS\anaconda3\Lib\site-packages\keras\src\optimizers\optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "C:\Users\THANOS\anaconda3\Lib\site-packages\keras\src\optimizers\optimizer.py", line 1223, in apply_gradients
       

In [ ]:
b_cnn_loss, b_cnn_accuracy = b_cnn.evaluate(x_test, y_test, verbose=0)
b_cnn_predictions = b_cnn.predict(x_test)
b_cnn_loss_v, b_cnn_accuracy_v = b_cnn.evaluate(x_validate, y_validate, verbose=0)
b_cnn_loss_t, b_cnn_accuracy_t = b_cnn.evaluate(x_train, y_train, verbose=0)
print("CNN Training: accuracy = %f" % (b_cnn_accuracy_t))
print("CNN Validation: accuracy = %f" % (b_cnn_accuracy_v))
print("CNN Test: accuracy = %f" % (b_cnn_accuracy))

In [ ]:
plt.plot(b_cnn_history.b_cnn_history['accuracy'])
plt.plot(b_cnn_history.b_cnn_history['val_accuracy'])
plt.title('CNN Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Test', 'Validation'])
plt.show()

In [ ]:
b_cnn_predictions = np.array(list(map(lambda x: np.argmax(x), b_cnn_predictions)))
categories = ['Actinic keratoses',
              'Basal cell carcinoma',
              'Benign keratosis-like lesions ', 
              'Dermatofibroma', 
              'Melanocytic nevi', 
              'Melanoma',
              'Vascular lesions']

CMatrix = pd.DataFrame(confusion_matrix(y_test_o, b_cnn_predictions), columns=categories, index =categories)
plt.figure(figsize=(12, 6)) 
ax = sns.heatmap(CMatrix, annot = True, fmt = 'g' ,vmin = 0, vmax = 10,cmap = 'bone_r') 
ax.set_xlabel('Predicted',fontsize = 14,weight = 'bold') 
ax.set_xticklabels(ax.get_xticklabels(),rotation =90); 
ax.set_ylabel('Actual',fontsize = 14,weight = 'bold')
ax.set_title('Confusion Matrix - Test Set',fontsize = 16,weight = 'bold',pad=20);
plt.show()

# AlexNet

In [12]:
#AlexNet Model

alexnet= keras.models.Sequential([
keras.layers.Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),activation='relu',
                    input_shape=(75,100,3)),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),activation='relu',padding="same"),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),activation='relu',padding="same"),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(filters=384,kernel_size=(1,1),strides=(1,1),activation='relu',padding="same"),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),activation='relu',padding="same"),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),keras.layers.Flatten(),
keras.layers.Dense(4096, activation='relu'),keras.layers.Dropout(0.5),
keras.layers.Dense(4096, activation='relu'),keras.layers.Dropout(0.5),
keras.layers.Dense(1000, activation='softmax')])
alexnet.compile(loss='sparse_categorical_crossentropy',optimizer=tensorflow.optimizers.SGD(learning_ra=0.001),
               metrics=['accuracy'])
alexnet.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 17, 23, 96)        34944     
                                                                 
 batch_normalization_5 (Bat  (None, 17, 23, 96)        384       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 8, 11, 96)         0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 8, 11, 256)        614656    
                                                                 
 batch_normalization_6 (Bat  (None, 8, 11, 256)        1024      
 chNormalization)                                                
                                                      

In [13]:
alexnet_history =alexnet.fit_generator(datagen.flow(x_train,y_train, batch_size=16),
                              epochs = 50, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // 16
                              , callbacks=[learning_rate_reduction])

C:\Users\THANOS\AppData\Local\Temp\ipykernel_11976\608476484.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  alexnet_history =alexnet.fit_generator(datagen.flow(x_train,y_train, batch_size=16),


Epoch 1/50



InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\THANOS\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\THANOS\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\THANOS\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start

  File "C:\Users\THANOS\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\THANOS\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\THANOS\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\THANOS\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\THANOS\

In [ ]:
alexnet_loss, alexnet_accuracy = alexnet.evaluate(x_test, y_test, verbose=0)
alexnet_predictions = alexnet.predict(x_test)
alexnet_loss_v, alexnet_accuracy_v = alexnet.evaluate(x_validate, y_validate, verbose=0)
alexnet_loss_t, alexnet_accuracy_t = alexnet.evaluate(x_train, y_train, verbose=0)
print("CNN Training: accuracy = %f" % (alexnet_accuracy_t))
print("CNN Validation: accuracy = %f" % (alexnet_accuracy_v))
print("CNN Test: accuracy = %f" % (alexnet_accuracy))

In [ ]:
plt.plot(alexnet_history.alexnet_history['accuracy'])
plt.plot(alexnet_history.alexnet_history['val_accuracy'])
plt.title('CNN Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Test', 'Validation'])
plt.show()

In [ ]:
alexnet_predictions = np.array(list(map(lambda x: np.argmax(x), alexnet_predictions)))
categories = ['Actinic keratoses',
              'Basal cell carcinoma',
              'Benign keratosis-like lesions ', 
              'Dermatofibroma', 
              'Melanocytic nevi', 
              'Melanoma',
              'Vascular lesions']

CMatrix = pd.DataFrame(confusion_matrix(y_test_o, alexnet_predictions), columns=categories, index =categories)
plt.figure(figsize=(12, 6)) 
ax = sns.heatmap(CMatrix, annot = True, fmt = 'g' ,vmin = 0, vmax = 10,cmap = 'bone_r') 
ax.set_xlabel('Predicted',fontsize = 14,weight = 'bold') 
ax.set_xticklabels(ax.get_xticklabels(),rotation =90); 
ax.set_ylabel('Actual',fontsize = 14,weight = 'bold')
ax.set_title('Confusion Matrix - Test Set',fontsize = 16,weight = 'bold',pad=20);
plt.show()

# ResNet (Transfer Learning)

In [1]:
import fastai
import pandas as pd
from fastai import *
from fastai.vision import *
from fastai.vision.all import *
from fastai.data.all import *

ModuleNotFoundError: No module named 'fastai'

In [ ]:
csv_path = path3
skin_df = pd.read_csv(csv_path)
skin_df.sort_values(by="image_id")


In [ ]:
lesion_type_dict = {'akiec': 'Actinic keratoses',
                    'bcc': 'Basal cell carcinoma',
                    'bkl': 'Benign keratosis-like lesions ',
                    'df': 'Dermatofibroma',
                    'nv': 'Melanocytic nevi',
                    'mel': 'Melanoma',
                    'vasc': 'Vascular lesions'}


In [ ]:
img_to_class_dict = skin_df.loc[:, ["image_id", "dx"]] 
img_to_class_dict = img_to_class_dict.to_dict('list') 
img_to_class_dict = {img_id : lesion_type_dict[disease] for img_id,disease in zip(img_to_class_dict['image_id'], img_to_class_dict['dx']) }
[x for x in img_to_class_dict.items()][:5]


In [ ]:
def get_label_from_dict(path):
    if 'path1' in str(path):  # Check if the path belongs to the first directory
        return img_to_class_dict_path1[path.stem]
    elif 'path2' in str(path):  # Check if the path belongs to the second directory
        return img_to_class_dict_path2[path.stem]

skin_db = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    item_tfms=[Resize(450), DihedralItem()],
    batch_tfms=RandomResizedCrop(size=224, min_scale=0.75, max_scale=1.0),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.3, seed=42),
    get_y=get_label_from_dict
)

img_path1 = path1  # Path to the first directory
img_path2 = path2  # Path to the second directory

# Get items from both directories and combine them
items_path1 = get_image_files(img_path1)
items_path2 = get_image_files(img_path2)
combined_items = items_path1 + items_path2

# Create DataLoader from combined items
dls = skin_db.dataloaders(combined_items)

In [ ]:
resnet = cnn_learner(dls, resnet18, metrics=accuracy, opt_func=ranger)
resnet.fine_tune(epochs=50, freeze_epochs=3, base_lr=0.005, cbs=MixUp(0.5))

# Xception

In [ ]:
from tensorflow.keras.applications import ResNet50
input_shape=(75,100,3)
baseModel = Xception(weights="imagenet", include_top=False,input_shape=input_shape)
#baseModel = ResNet50(weights="imagenet", include_top=False,
#input_tensor=Input(shape=(75,100,3)))
#baseModel = Xception(weights="imagenet", include_top=False,
#input_tensor=Input(shape=(75,100,3)))
#baseModel = EfficientNetB4(weights="imagenet", include_top=False,
#input_tensor=Input(shape=(75,100,3)))
xcptn = baseModel.output
xcptn = Flatten()(xcptn)
xcptn = Dense(128, activation="relu")(xcptn)
xcptn = Dropout(0.2)(xcptn)
xcptn = Dense(num_classes, activation="softmax")(xcptn)
xcptn = Model(inputs=baseModel.input, outputs=xcptn)
xcptn.summary()
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
xcptn.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
xcptn_history = xcptn.fit_generator(datagen.flow(x_train,y_train, batch_size=20),
                              epochs = 50, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // 20
                              , callbacks=[learning_rate_reduction])

In [ ]:
xcptn_loss, xcptn_accuracy = xcptn.evaluate(x_test, y_test, verbose=0)
xcptn_predictions = xcptn.predict(x_test)
xcptn_loss_v, xcptn_accuracy_v = xcptn.evaluate(x_validate, y_validate, verbose=0)
xcptn_loss_t, xcptn_accuracy_t = xcptn.evaluate(x_train, y_train, verbose=0)
print("Training: accuracy = %f" % (xcptn_accuracy_t))
print("Validation: accuracy = %f" % (xcptn_accuracy_v))
print("Test: accuracy = %f" % (xcptn_accuracy))

In [ ]:
plt.plot(xcptn_history.xcptn_history['accuracy'])
plt.plot(xcptn_history.xcptn_history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Test', 'Validation'])
plt.show()

In [ ]:
xcptn_predictions = np.array(list(map(lambda x: np.argmax(x), xcptn_predictions)))
categories = ['Actinic keratoses',
              'Basal cell carcinoma',
              'Benign keratosis-like lesions ', 
              'Dermatofibroma', 
              'Melanocytic nevi', 
              'Melanoma',
              'Vascular lesions']

CMatrix = pd.DataFrame(confusion_matrix(y_test_o, xcptn_predictions), columns=categories, index =categories)
plt.figure(figsize=(12, 6)) 
ax = sns.heatmap(CMatrix, annot = True, fmt = 'g' ,vmin = 0, vmax = 10,cmap = 'bone_r') 
ax.set_xlabel('Predicted',fontsize = 14,weight = 'bold') 
ax.set_xticklabels(ax.get_xticklabels(),rotation =90); 
ax.set_ylabel('Actual',fontsize = 14,weight = 'bold')
ax.set_title('Confusion Matrix - Test Set',fontsize = 16,weight = 'bold',pad=20);
plt.show()

*  #  High Fusion (Decision Fusion)

In [ ]:
# 1) Majority Voting
"""--Hard Voting: Predict the class with the largest sum of votes from models
   --Soft Voting: Predict the class with the largest summed probability from models"""
fusion_1_hard = VotingClassifier(estimators=[('Basic CNN', b_cnn), ('AlexNet', alexnet), ('ResNet', resnet), ('Xception', xception)], voting='hard')
fusion_1_hard.fit(x_train, y_train)
fusion_1_hard.predict(x_test)  # Check sklearn documentation for more information

fusion_1_soft = VotingClassifier(estimators=[('Basic CNN', b_cnn), ('AlexNet', alexnet), ('ResNet', resnet), ('Xception', xception)], voting='soft')
fusion_1_soft.fit(x_train, y_train)
fusion_1_soft.predict(x_test)  # Check sklearn documentation for more information

In [ ]:
# 2) Weighted Voting
weights = {'Basic CNN': 1, 'AlexNet': 2, 'ResNet': 3, 'Xception': 4 }  # Example weights, adjust as needed
fusion_2_hard_weighted = VotingClassifier(estimators=[('Basic CNN', b_cnn), ('AlexNet', alexnet), ('ResNet', resnet), ('Xception', xception)], voting='hard', weights=weights)
fusion_2_hard_weighted.fit(X_train, y_train)
fusion_2_hard_weighted.predict(X_test)

weights = {'Basic CNN': 1, 'AlexNet': 2, 'ResNet': 3, 'Xception': 4 }  # Example weights, adjust as needed
fusion_2_soft_weighted = VotingClassifier(estimators=[('Basic CNN', b_cnn), ('AlexNet', alexnet), ('ResNet', resnet), ('Xception', xception)], voting='soft', weights=weights)
fusion_2_soft_weighted.fit(X_train, y_train)
fusion_2_soft_weighted.predict(X_test)

# TODO: how to select the weights?

In [ ]:
# 3) Bayesian Consensus
classifiers = [b_cnn, alexnet, resnet, xception] 

class_probs = []
for clf in classifiers:
    prob = clf.predict_proba(X_test)  
    class_probs.append(prob)

consensus_probs = np.prod(class_probs, axis=0) ** (1 / len(classifiers))
final_predictions = np.argmax(consensus_probs, axis=1)

In [ ]:
# 4) Dempster-Shafer's Theory of evidence
from pyds import MassFunction
classifiers = [b_cnn, alexnet, resnet, xception]  

predictions = []
for clf in classifiers:
    pred = clf.predict(X_test)
    predictions.append(pred)
    
bpas = [MassFunction({cls: confidence for cls, confidence in zip(np.unique(pred), clf.predict_proba(X_test))}) for pred, clf in zip(predictions, classifiers)]

combined_bpa = bpas[0]
for bpa in bpas[1:]:
    combined_bpa = combined_bpa.combine(bpa)

final_decision = combined_bpa.argmax()

In [ ]:
# TODO: Evaluate and Report performance of classifiers before and after applying fusion